In [ ]:
import math
import numpy as np
import stats_getter
from stats_getter import get_league_game_log, get_team_id, get_team_pt_shots, get_team_shot_locations, getLeagueDashPlayerStats, getLeagueDashTeamStats
import pandas as pd
from datetime import datetime, timedelta
from nba_api.stats.endpoints import LeagueDashTeamShotLocations
from nba_api.stats.static import teams
import importlib
importlib.reload(stats_getter)
from typing import Dict
import cache_manager
from cache_manager import stats_cache

import time
import random


def get_hardcoded_2014_2015_averages() -> Dict[str, float]:
    """
    Precisely calculated league averages from 2014-2015 season data.
    Uses the helper functions to get actual NBA API data for all required statistics.
    """
    # Season and date range for 2014-2015
    season = "2014-15"
    date_from = "10/28/2014"  # Start of 2014-15 season
    date_to = "04/15/2015"    # End of regular season 2014-15
    
    # All 30 NBA teams for 2014-15 season
    test_teams = [
        "Atlanta Hawks", "Boston Celtics", "Brooklyn Nets", "Charlotte Hornets", "Chicago Bulls",
        "Cleveland Cavaliers", "Dallas Mavericks", "Denver Nuggets", "Detroit Pistons", "Golden State Warriors",
        "Houston Rockets", "Indiana Pacers", "Los Angeles Clippers", "Los Angeles Lakers", "Memphis Grizzlies",
        "Miami Heat", "Milwaukee Bucks", "Minnesota Timberwolves", "New Orleans Pelicans", "New York Knicks",
        "Oklahoma City Thunder", "Orlando Magic", "Philadelphia 76ers", "Phoenix Suns", "Portland Trail Blazers",
        "Sacramento Kings", "San Antonio Spurs", "Toronto Raptors", "Utah Jazz", "Washington Wizards"
    ]
    
    # Initialize lists to store all team values
    ast_ratios = []
    tov_pcts = []
    contested3_fg_pcts = []
    contested2_fg_pcts = []
    corner3_fg_pcts = []
    above_break3_fg_pcts = []
    paint_fg_pcts = []
    mid_range_fg_pcts = []
    pts_off_tov = []
    
    print(f"Processing {len(test_teams)} teams...")
    
    # Get data for each team
    for i, team_name in enumerate(test_teams):
        print(f"Processing {team_name} ({i+1}/{len(test_teams)})...")
        
        try:
            # Add delay between requests to avoid rate limiting
            if i > 0:
                delay = random.uniform(2, 4)  # Random delay 2-4 seconds
                print(f"  Waiting {delay:.1f}s to avoid rate limits...")
                time.sleep(delay)
            
            # 1. Get AST_RATIO and TOV_PCT from Advanced stats
            print("  Getting advanced stats...")
            try:
                # Try without date filtering first for older seasons
                advanced_stats = getLeagueDashTeamStats(
                    team_name=team_name,
                    season=season,
                    date_from=None,  # Remove date filtering for older data
                    date_to=None,    # Remove date filtering for older data
                    measure_type="Advanced",
                    per_mode="PerGame"
                )
                
                if not advanced_stats.empty:
                    # Check available columns
                    print(f"    Available columns: {list(advanced_stats.columns)}")
                    
                    # Try different column names for turnover percentage
                    ast_ratio = advanced_stats['AST_RATIO'].iloc[0] if 'AST_RATIO' in advanced_stats.columns else None
                    
                    # Try different column names for turnover percentage
                    tov_pct = None
                    for col in ['TM_TOV_PCT', 'TOV_PCT', 'TEAM_TOV_PCT']:
                        if col in advanced_stats.columns:
                            tov_pct = advanced_stats[col].iloc[0]
                            print(f"    Found turnover column: {col}")
                            break
                    
                    if ast_ratio is not None:
                        ast_ratios.append(ast_ratio)
                        print(f"    AST_RATIO: {ast_ratio}")
                    
                    if tov_pct is not None:
                        # Keep TOV_PCT as is - it's already in the correct format
                        tov_pcts.append(tov_pct)
                        print(f"    TOV_PCT: {tov_pct:.3f}")
                
            except Exception as e:
                print(f"    Error getting advanced stats: {e}")
            
            # Add delay between different endpoint calls
            time.sleep(3)  # Longer delay between endpoints
            
            # 2. Get PTS_OFF_TOV from Misc stats
            print("  Getting misc stats...")
            try:
                misc_stats = getLeagueDashTeamStats(
                    team_name=team_name,
                    season=season,
                    date_from=None,  # Remove date filtering
                    date_to=None,    # Remove date filtering
                    measure_type="Misc",
                    per_mode="PerGame"
                )
                
                if not misc_stats.empty and 'PTS_OFF_TOV' in misc_stats.columns:
                    pts_off_turnovers = misc_stats['PTS_OFF_TOV'].iloc[0]
                    pts_off_tov.append(pts_off_turnovers)
                    print(f"    PTS_OFF_TOV: {pts_off_turnovers}")
                
            except Exception as e:
                print(f"    Error getting misc stats: {e}")
            
            time.sleep(3)  # Longer delay between endpoints
            
            # 3. Try to get contested shot data (this might fail for older seasons)
            print("  Getting contested shot data...")
            try:
                pt_shots_list = get_team_pt_shots(
                    team_name=team_name,
                    season=season,
                    date_from=None,  # Remove date filtering
                    date_to=None,    # Remove date filtering
                    per_mode="PerGame"
                )
                
                # pt_shots_list should be a list of DataFrames
                if pt_shots_list and len(pt_shots_list) > 3:
                    df_team = pt_shots_list[3]  # Team's own shooting data
                    print(f"    Team shot data shape: {df_team.shape}")
                    
                    # Check if we have the defender distance column
                    if 'CLOSE_DEF_DIST_RANGE' in df_team.columns:
                        # Filter for contested shots
                        very_tight = df_team[df_team['CLOSE_DEF_DIST_RANGE'] == '0-2 Feet - Very Tight']
                        tight = df_team[df_team['CLOSE_DEF_DIST_RANGE'] == '2-4 Feet - Tight']
                        
                        # Get contested 3PT data
                        if len(very_tight) > 0 or len(tight) > 0:
                            # For 3PT shots, use FG3A and FG3M directly
                            contested3_makes = 0
                            contested3_attempts = 0
                            
                            if len(very_tight) > 0:
                                contested3_makes += very_tight['FG3M'].iloc[0]
                                contested3_attempts += very_tight['FG3A'].iloc[0]
                            
                            if len(tight) > 0:
                                contested3_makes += tight['FG3M'].iloc[0]
                                contested3_attempts += tight['FG3A'].iloc[0]
                            
                            if contested3_attempts > 0:
                                contested3_pct = (contested3_makes / contested3_attempts) * 100
                                contested3_fg_pcts.append(contested3_pct)
                                print(f"    Contested 3PT%: {contested3_pct:.1f}%")
                        
                        # Get contested 2PT data (FGA - FG3A = 2PT attempts)
                        if len(very_tight) > 0 or len(tight) > 0:
                            contested2_makes = 0
                            contested2_attempts = 0
                            
                            if len(very_tight) > 0:
                                vt_2pt_makes = very_tight['FGM'].iloc[0] - very_tight['FG3M'].iloc[0]
                                vt_2pt_attempts = very_tight['FGA'].iloc[0] - very_tight['FG3A'].iloc[0]
                                contested2_makes += vt_2pt_makes
                                contested2_attempts += vt_2pt_attempts
                            
                            if len(tight) > 0:
                                t_2pt_makes = tight['FGM'].iloc[0] - tight['FG3M'].iloc[0]
                                t_2pt_attempts = tight['FGA'].iloc[0] - tight['FG3A'].iloc[0]
                                contested2_makes += t_2pt_makes
                                contested2_attempts += t_2pt_attempts
                            
                            if contested2_attempts > 0:
                                contested2_pct = (contested2_makes / contested2_attempts) * 100
                                contested2_fg_pcts.append(contested2_pct)
                                print(f"    Contested 2PT%: {contested2_pct:.1f}%")
                    else:
                        print("    No CLOSE_DEF_DIST_RANGE column found")
                else:
                    print("    pt_shots returned unexpected format")
                
            except Exception as e:
                print(f"    Error getting shot data: {e}")
            
            time.sleep(1)
            
            # 4. Get shot location data - FIXED VERSION
            print("  Getting shot location data...")
            try:
                df_locations = get_team_shot_locations(
                    team_name=team_name,
                    season=season,
                    date_from=None,
                    date_to=None,
                    measure_type="Base"
                )
                
                if df_locations is not None and not df_locations.empty:
                    print(f"    Location data shape: {df_locations.shape}")
                    
                    # Parse corner 3PT shots - CORRECTED ACCESS METHOD
                    try:
                        # Use direct column access with tuple, not .loc
                        left_corner_fgm = df_locations[('Left Corner 3', 'FGM')].iloc[0]
                        left_corner_fga = df_locations[('Left Corner 3', 'FGA')].iloc[0]
                        right_corner_fgm = df_locations[('Right Corner 3', 'FGM')].iloc[0]
                        right_corner_fga = df_locations[('Right Corner 3', 'FGA')].iloc[0]
                        
                        total_corner_fga = left_corner_fga + right_corner_fga
                        total_corner_fgm = left_corner_fgm + right_corner_fgm
                        
                        if total_corner_fga > 0:
                            corner3_fg_pct = (total_corner_fgm / total_corner_fga) * 100
                            corner3_fg_pcts.append(corner3_fg_pct)
                            print(f"    Corner 3PT: {corner3_fg_pct:.1f}% ({total_corner_fgm}/{total_corner_fga})")
                    except (KeyError, IndexError) as e:
                        print(f"    Corner 3PT data not available: {e}")
                    
                    # Parse above the break 3PT shots - CORRECTED
                    try:
                        above_break_fgm = df_locations[('Above the Break 3', 'FGM')].iloc[0]
                        above_break_fga = df_locations[('Above the Break 3', 'FGA')].iloc[0]
                        
                        if above_break_fga > 0:
                            above_break3_fg_pct = (above_break_fgm / above_break_fga) * 100
                            above_break3_fg_pcts.append(above_break3_fg_pct)
                            print(f"    Above Break 3PT: {above_break3_fg_pct:.1f}% ({above_break_fgm}/{above_break_fga})")
                    except (KeyError, IndexError) as e:
                        print(f"    Above Break 3PT data not available: {e}")
                    
                    # Parse paint and restricted area stats - CORRECTED
                    try:
                        paint_fgm = df_locations[('In The Paint (Non-RA)', 'FGM')].iloc[0]
                        paint_fga = df_locations[('In The Paint (Non-RA)', 'FGA')].iloc[0]
                        restricted_fgm = df_locations[('Restricted Area', 'FGM')].iloc[0]
                        restricted_fga = df_locations[('Restricted Area', 'FGA')].iloc[0]
                        
                        total_paint_fga = paint_fga + restricted_fga
                        total_paint_fgm = paint_fgm + restricted_fgm
                        
                        if total_paint_fga > 0:
                            paint_fg_pct = (total_paint_fgm / total_paint_fga) * 100
                            paint_fg_pcts.append(paint_fg_pct)
                            print(f"    Paint: {paint_fg_pct:.1f}% ({total_paint_fgm}/{total_paint_fga})")
                    except (KeyError, IndexError) as e:
                        print(f"    Paint data not available: {e}")
                    
                    # Parse mid-range shots - CORRECTED
                    try:
                        mid_range_fgm = df_locations[('Mid-Range', 'FGM')].iloc[0]
                        mid_range_fga = df_locations[('Mid-Range', 'FGA')].iloc[0]
                        
                        if mid_range_fga > 0:
                            mid_range_fg_pct = (mid_range_fgm / mid_range_fga) * 100
                            mid_range_fg_pcts.append(mid_range_fg_pct)
                            print(f"    Mid-Range: {mid_range_fg_pct:.1f}% ({mid_range_fgm}/{mid_range_fga})")
                    except (KeyError, IndexError) as e:
                        print(f"    Mid-Range data not available: {e}")
                
            except Exception as e:
                print(f"    Error getting location data: {e}")
            
            print(f"  ✅ Completed {team_name}")
            
        except Exception as e:
            print(f"❌ Error processing {team_name}: {e}")
            continue
    
    # Calculate averages
    def safe_average(values):
        return sum(values) / len(values) if values else 0.0
    
    print("\n📊 Final Results:")
    
    result = {
        "AST_RATIO": safe_average(ast_ratios),
        "TOV_PCT": safe_average(tov_pcts),
        "contested3_FG_PCT": safe_average(contested3_fg_pcts),
        "contested2_FG_PCT": safe_average(contested2_fg_pcts),
        "corner3_FG_PCT": safe_average(corner3_fg_pcts),
        "aboveBreak3_FG_PCT": safe_average(above_break3_fg_pcts),
        "paint_FG_PCT": safe_average(paint_fg_pcts),
        "midRange_FG_PCT": safe_average(mid_range_fg_pcts),
        "PTS_OFF_TOV": safe_average(pts_off_tov)
    }
    
    # Print sample sizes
    print(f"Sample sizes:")
    print(f"  AST_RATIO: {len(ast_ratios)} teams")
    print(f"  TOV_PCT: {len(tov_pcts)} teams")
    print(f"  contested3_FG_PCT: {len(contested3_fg_pcts)} values")
    print(f"  contested2_FG_PCT: {len(contested2_fg_pcts)} values")
    print(f"  PTS_OFF_TOV: {len(pts_off_tov)} teams")
    
    return result

def get_league_average_stat(stat_name: str,
                          season: str,
                          csv_path: str = "nba_complete_dataset_20152025_final.csv") -> float:
    """
    Get league average for specific stats using previous season's data.
    
    Parameters:
    - stat_name: One of the required stats
    - season: Target season in format "YYYY-YYYY"
    - csv_path: Path to the CSV file
    
    Stats supported:
    - AST_RATIO: assist ratio
    - TOV_PCT: turnover ratio
    - contested3_FG_PCT: Contested 3P%
    - open3_FG_PCT: Open 3P%
    - aboveBreak3_FG_PCT: Above the break 3P%
    - corner3_FG_PCT: Corner 3P%
    - paint_FG_PCT: Paint FG%
    - midRange_FG_PCT: Mid Range FG%
    - restrictedArea_FG_PCT: Restricted Area FG%
    - contested2_FG_PCT: Contested 2P%
    - PTS_OFF_TOV: Points off Turnovers
    - PTS_FB: Points on fast breaks
    """
    
    # For 2015-16 season, use hardcoded 2014-15 data
    if season == "2015-16":
        hardcoded_2014_15_averages = {
            "AST_RATIO": 11.4,
            "TOV_PCT": 0.1835,
            "contested3_FG_PCT": 32.30443866890893,
            "contested2_FG_PCT": 50.540873322971805,
            "corner3_FG_PCT": 43.709884000174654,
            "aboveBreak3_FG_PCT": 36.71466628530988,
            "paint_FG_PCT": 56.12985931951518,
            "midRange_FG_PCT": 40.715821469602076,
            "PTS_OFF_TOV": 19.95,
            "PTS_FB": 9.016666666666664
        }
        
        if stat_name in hardcoded_2014_15_averages:
            return hardcoded_2014_15_averages[stat_name]
    
    # For other seasons, read from CSV
    start_year = int(season.split("-")[0])
    prev_season_start = start_year - 1
    prev_season = f"{prev_season_start}-{str(start_year)[-2:]}"
    
    # Read CSV
    df = pd.read_csv(csv_path)
    df['season'] = df['season'].astype(str)
    df_prev = df[df['season'] == prev_season].copy()
    
    if df_prev.empty:
        raise ValueError(f"No data found for season {prev_season}")
    
    # Get last game for each team (final cumulative averages)
    df_prev['date'] = pd.to_datetime(df_prev['date'])
    df_prev = df_prev.sort_values('date')
    
    # Get final stats for each team
    home_final = df_prev.groupby('home_team').last()
    away_final = df_prev.groupby('away_team').last()
    
    # Collect values for the stat
    values = []
    
    # Check for home_ and away_ prefixed columns
    home_col = f"home_{stat_name}"
    away_col = f"away_{stat_name}"
    
    if home_col in home_final.columns:
        values.extend(home_final[home_col].dropna().tolist())
    if away_col in away_final.columns:
        values.extend(away_final[away_col].dropna().tolist())
    
    if not values:
        raise ValueError(f"No data found for stat {stat_name}")
    
    return float(np.mean(values))


# Convenience function to get all required averages
def get_all_required_league_averages(season: str, 
                                    csv_path: str = "nba_complete_dataset_20152025_final.csv") -> Dict[str, float]:
    """
    Get all the specific league averages needed for the PickPocket project.
    """
    required_stats = [
        "AST_RATIO",           # assists generated every 100 poss
        "TOV_PCT",             # turnover ratio
        "contested3_FG_PCT",   # Contested 3P%
        "open3_FG_PCT",        # Open 3P%
        "aboveBreak3_FG_PCT",  # Above the break 3P%
        "corner3_FG_PCT",      # Corner 3P%
        "paint_FG_PCT",        # Paint FG%
        "midRange_FG_PCT",     # Mid Range FG%
        "restrictedArea_FG_PCT", # Restricted Area FG%
        "contested2_FG_PCT",   # Contested 2P%
        "PTS_OFF_TOV",         # Points off Turnovers
        "PTS_FB"               # Points on fast breaks
    ]
    
    averages = {}
    for stat in required_stats:
        try:
            averages[stat] = get_league_average_stat(stat, season, csv_path)
        except Exception as e:
            print(f"Warning: Could not get {stat}: {e}")
    
    return averages


def getB2B(team: str, season: str, date: str) -> int:
    """
    Returns 1 if the game on `date` was a back-to-back for `team`, else 0.

    Parameters:
      team   (str): e.g. "Portland Trail Blazers"
      season (str): e.g. "2019-20"
      date   (str): "MM/DD/YYYY", e.g. "03/07/2020"
    """
    df = stats_getter.get_game_log(team, season)

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='%b %d, %Y')

    df = df.sort_values('GAME_DATE').reset_index(drop=True)

    try:
        target = datetime.strptime(date, "%m/%d/%Y")
    except ValueError:
        raise ValueError("`date` must be in MM/DD/YYYY format")

    mask = df['GAME_DATE'] == target
    if not mask.any():
        raise ValueError(f"No {team} game found on {date} in {season}")
    idx = df.index[mask][0]

    if idx == 0:
        return 0

    prev_game = df.loc[idx-1, 'GAME_DATE']
    return int((target - prev_game).days == 1)

def getStocksDeflectionsDiff(team_name: str, season: str, date: str) -> float:
    """
    Computes (steals + blocks + deflections) per game differential
    for team_name vs. its opponent on `date` in season.

    Raises ValueError if team_name has no game on `date`.
    """
    # 1) parse the game date
    dte = datetime.strptime(date, "%m/%d/%Y")
    target_dt = dte - timedelta(days=1)

    # 2) pull the full league game log (your helper returns a DataFrame)
    df_lg = stats_getter.get_league_game_log(season)
    df_lg["GAME_DATE"] = pd.to_datetime(df_lg["GAME_DATE"])  # ISO format

    # 3) get the team ID via your helper
    team_id = stats_getter.get_team_id(team_name)
    if team_id is None:
        raise ValueError(f"Unknown team: {team_name}")

    # 4) check that they actually played on target_dt
    played = ((df_lg["TEAM_ID"] == team_id) &
              (df_lg["GAME_DATE"] == target_dt)).any()
    if not played:
        raise ValueError(f"{team_name} did not play on {date}")

    # 5) keep all games up to & including the target date
    df_up = df_lg[df_lg["GAME_DATE"] < target_dt]

    # 6) compute Team A's steals & blocks per game
    dfA  = df_up[df_up["TEAM_ID"] == team_id]
    spgA = dfA["STL"].mean()
    bpgA = dfA["BLK"].mean()

    # 7) pull Team A's deflections per game
    
    start_str = dfA["GAME_DATE"].min().strftime("%Y/%m/%d")
    hustle_df = stats_getter.getLeagueHustleTeamStats(
        team_id_nullable=team_id, 
        season=season,
        date_to_nullable=target_dt,
        per_mode = "PerGame"

    )
    defA = hustle_df.loc[hustle_df["TEAM_ID"] == team_id, "DEFLECTIONS"].item()
    
    # 8) find opponent on that date
    matchup  = df_lg.loc[
                   (df_lg["TEAM_ID"] == team_id) &
                   (df_lg["GAME_DATE"] == target_dt),
                   "MATCHUP"
               ].iloc[0]
    opp_abbr = matchup.split()[-1]
    opp_full = next(t["full_name"] for t in teams.get_teams()
                    if t["abbreviation"] == opp_abbr)
    opp_id   = stats_getter.get_team_id(opp_full)

    # 9) compute Opponent's SPG & BPG
    dfB  = df_up[df_up["TEAM_ID"] == opp_id]
    spgB = dfB["STL"].mean()
    bpgB = dfB["BLK"].mean()

    # 10) pull Opponent’s deflections
    defB = hustle_df.loc[hustle_df["TEAM_ID"] == opp_id,  "DEFLECTIONS"].item()

    # 11) return the differential
    return (spgA + bpgA + defA) - (spgB + bpgB + defB)





def get_pf_diff(team_a_name, team_b_name, season, date_str):
    """
    Computes the difference in average personal fouls per game between two teams
    up to (and including) a given date in a season.

    Parameters:
        team_a_name (str): Full name of Team A (e.g., 'Los Angeles Lakers')
        team_b_name (str): Full name of Team B (e.g., 'Boston Celtics')
        season (str): NBA season formatted like '2023-24'
        date_str (str): Date in "MM/DD/YYYY" format (e.g., "12/01/2023")

    Returns:
        float: A_pf_per_game - B_pf_per_game (up to that date)
    """

    team_a_id = get_team_id(team_a_name)
    team_b_id = get_team_id(team_b_name)
    target_date = datetime.strptime(date_str, "%m/%d/%Y")

    # Load full league game log
    df = get_league_game_log(season)

    # Convert GAME_DATE column to datetime
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

    # Filter up to the target date
    df = df[df['GAME_DATE'] < target_date]

    # Compute PF per game up to that date for both teams
    team_a_pf = df[df['TEAM_ID'] == team_a_id]['PF'].mean()
    team_b_pf = df[df['TEAM_ID'] == team_b_id]['PF'].mean()

    return float(team_a_pf - team_b_pf)

def get_pace_diff(team_a_name, team_b_name, season, date_str):
    """
    Returns (PACE_A)/(PACE_B), where each PACE is the team's average
    pace per game from the start of the season through the day before date_str.

    All dates are MM/DD/YYYY strings.

    Raises ValueError if either team has no games in that window.
    """
    # 1) parse the target and compute the day‑before cutoff
    target     = datetime.strptime(date_str, "%m/%d/%Y")
    day_before = target - timedelta(days=1)
    date_to    = day_before.strftime("%m/%d/%Y")    # e.g. "10/21/2019"

    # 2) season start = October 1 of the first year
    first_year = int(season.split("-")[0])
    date_from  = f"10/01/{first_year}"              # e.g. "10/01/2019"

    # 3) pull Team A’s advanced per‑game stats in [date_from … date_to]
    dfA = stats_getter.getLeagueDashTeamStats(
        team_a_name,
        season,
        date_from,
        date_to,
        measure_type="Advanced",
        per_mode   ="PerGame",
    )
    if dfA.empty:
        raise ValueError(f"{team_a_name} had no games by {date_to}")

    # 4) same for Team B
    dfB = stats_getter.getLeagueDashTeamStats(
        team_b_name,
        season,
        date_from,
        date_to,
        measure_type="Advanced",
        per_mode   ="PerGame",
    )
    if dfB.empty:
        raise ValueError(f"{team_b_name} had no games by {date_to}")

    # 5) extract each team’s PACE (there will be exactly one row per df)
    pace_a = dfA["PACE"].iloc[0]
    pace_b = dfB["PACE"].iloc[0]

    # 6) return the ratio
    return pace_a / pace_b

def get_clutch_netrtg_ratio(team_a_name: str,
                            team_b_name: str,
                            season: str,
                            date_str: str) -> float:
    """
    Returns A_clutch_NETRTG / B_clutch_NETRTG for a given season up through
    the day before `date_str`.  If either team has no clutch data in that
    window, returns math.nan.
    """
    # 1) parse the target date & compute cutoff (day before)
    target   = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff   = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    # 2) season start = October 1 of the first year
    start_yr = int(season.split("-")[0])
    date_from= f"10/01/{start_yr}"

    # 3) helper to fetch one team’s clutch NET_RATING or None
    def _fetch_netrtg(team_name):
        df = stats_getter.getLeagueDashTeamClutch(
            team_name,
            season,
            date_from,
            cutoff,
            measure_type="Advanced",
            per_mode="PerGame"
        )
        return df["NET_RATING"].iloc[0] if not df.empty else None

    netA = _fetch_netrtg(team_a_name)
    netB = _fetch_netrtg(team_b_name)

    # 4) if either is missing, return NaN
    if netA is None or netB is None:
        return math.nan

    # 5) otherwise return the ratio
    return netA / netB

def get_fta_rate_relative(team_a_name: str,
                          team_b_name: str,
                          season: str,
                          date_str: str) -> float:
    """
    A_FTA_RATE * (B_opp_FTA_RATE / league_avg_FTA_RATE),
    using all games *before* date_str (exclusive).
    """
    # 1) parse cutoff
    target_dt = datetime.strptime(date_str, "%m/%d/%Y")

    # 2) pull & filter by date-only
    df = stats_getter.get_league_game_log(season).copy()
    df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
    df_before    = df[df["GAME_DATE"] < target_dt]
    if df_before.empty:
        raise ValueError(f"No games before {date_str} in {season}")

    # 3) get IDs
    a_id = stats_getter.get_team_id(team_a_name)
    b_id = stats_getter.get_team_id(team_b_name)
    if a_id is None or b_id is None:
        raise ValueError("Unknown team")

    # 4) A_FTA_RATE
    dfA = df_before[df_before["TEAM_ID"] == a_id]
    if dfA.empty:
        raise ValueError(f"{team_a_name} has no games before {date_str}")
    fta_rate_A = dfA["FTA"].sum() / dfA["FGA"].sum()

    # 5) B_opp_FTA_RATE
    dfB       = df_before[df_before["TEAM_ID"] == b_id]
    opp_rows  = df_before[
        df_before["GAME_ID"].isin(dfB["GAME_ID"]) &
        (df_before["TEAM_ID"] != b_id)
    ]
    if opp_rows.empty:
        raise ValueError(f"No opponent stats for {team_b_name} by {date_str}")
    fta_rate_Bopp = opp_rows["FTA"].sum() / opp_rows["FGA"].sum()

    # 6) league avg
    league_avg = df_before["FTA"].sum() / df_before["FGA"].sum()

    # 7) combine
    return fta_rate_A * (fta_rate_Bopp / league_avg)


def get_pfd_diff(team_a_name: str,
                 team_b_name: str,
                 season: str,
                 date_str: str) -> float:
    """
    Returns the difference in personal‐fouls‐drawn per game (A_pfd − B_pfd)
    for Team A vs. Team B over all games *before* the date_str.

    Uses your getLeagueDashTeamStats helper with:
      • measure_type="Misc"
      • per_mode="PerGame"

    Parameters:
      team_a_name (str): e.g. "Portland Trail Blazers"
      team_b_name (str): e.g. "Los Angeles Lakers"
      season       (str): "2019-20"
      date_str     (str): target date "MM/DD/YYYY"

    Returns:
      float: A_pfd_per_game − B_pfd_per_game
    """
    # 1) parse the cutoff date and step back one day
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff_dt = target - timedelta(days=1)
    date_to   = cutoff_dt.strftime("%m/%d/%Y")

    # 2) season start = October 1 of the first year
    start_year = int(season.split("-")[0])
    date_from  = f"10/01/{start_year}"

    # 3) fetch Team A's misc per‑game stats up through cutoff
    dfA = stats_getter.getLeagueDashTeamStats(
        team_a_name,
        season,
        date_from,
        date_to,
        measure_type="Base",
        per_mode="PerGame"
    )
    if dfA.empty:
        raise ValueError(f"{team_a_name} had no misc data by {date_to}")
    pfdA = dfA["PFD"].iloc[0]  # personal fouls drawn per game

    # 4) fetch Team B's misc per‑game stats
    dfB = stats_getter.getLeagueDashTeamStats(
        team_b_name,
        season,
        date_from,
        date_to,
        measure_type="Base",
        per_mode="PerGame"
    )
    if dfB.empty:
        raise ValueError(f"{team_b_name} had no misc data by {date_to}")
    pfdB = dfB["PFD"].iloc[0]

    # 5) return the difference
    return pfdA - pfdB


def get_dreb_pct_relative(team_a_name: str,
                          team_b_name: str,
                          season: str,
                          date_str: str) -> float:
    """
    Returns A_DREB_PCT * (B_opp_DREB_PCT / LeagueAvg_DREB_PCT),
    where all stats are up through (but not including) date_str.

    - A_DREB_PCT from Advanced PerGame
    - B_opp_DREB_PCT computed as OPP_DREB/(OPP_DREB+OPP_OREB) from Opponent PerGame
    - LeagueAvg_DREB_PCT from summing raw DREB/OREB in the league log
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"

    # 2) A_DREB_PCT from Advanced
    dfA = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA.empty:
        return math.nan
    a_pct = dfA["DREB_PCT"].iloc[0]

    # 3) pull OPP_DREB & OPP_OREB for Team B
    dfBopp = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Opponent", per_mode="PerGame"
    )
    if dfBopp.empty:
        return math.nan

    opp_dreb = dfBopp["OPP_DREB"].iloc[0]
    opp_oreb = dfBopp["OPP_OREB"].iloc[0]
    total    = opp_dreb + opp_oreb
    if total == 0:
        return math.nan
    b_opp_pct = opp_dreb / total

    # 4) league‐wide DREB / (DREB+OREB)
    df = stats_getter.get_league_game_log(season).copy()
    df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
    df_cut = df[df["GAME_DATE"] <= datetime.strptime(cutoff, "%m/%d/%Y")]
    if df_cut.empty:
        return math.nan
    league_dreb = df_cut["DREB"].sum()
    league_oreb = df_cut["OREB"].sum()
    if league_dreb + league_oreb == 0:
        return math.nan
    league_avg = league_dreb / (league_dreb + league_oreb)

    # 5) combine
    return a_pct * (b_opp_pct / league_avg)

def get_oreb_pct_relative(team_a_name: str,
                          team_b_name: str,
                          season: str,
                          date_str: str) -> float:
    """
    Returns A_OREB_PCT * (B_opp_OREB_PCT / LeagueAvg_OREB_PCT),
    where all stats are up through (but not including) date_str.

    - A_OREB_PCT from Advanced PerGame
    - B_opp_OREB_PCT computed as B_opp_OREB/(B_opp_OREB+B_DREB) from Opponent PerGame and Base PerGame
    - LeagueAvg_OREB_PCT from summing raw OREB/DREB in the league log
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"

    # 2) A_OREB_PCT from Advanced
    dfA = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA.empty:
        return math.nan
    a_pct = dfA["OREB_PCT"].iloc[0]

    # 3) pull OPP_OREB for Team B (from Opponent stats)
    dfBopp = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Opponent", per_mode="PerGame"
    )
    if dfBopp.empty:
        return math.nan
    opp_oreb = dfBopp["OPP_OREB"].iloc[0]

    # 4) pull B_DREB for Team B (from Base stats)
    dfBbase = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Base", per_mode="PerGame"
    )
    if dfBbase.empty:
        return math.nan
    b_dreb = dfBbase["DREB"].iloc[0]

    # Calculate B_opp_OREB_PCT
    total = opp_oreb + b_dreb
    if total == 0:
        return math.nan
    b_opp_pct = opp_oreb / total

    # 5) league‐wide OREB / (OREB+DREB)
    df = stats_getter.get_league_game_log(season).copy()
    df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
    df_cut = df[df["GAME_DATE"] <= datetime.strptime(cutoff, "%m/%d/%Y")]
    if df_cut.empty:
        return math.nan
    league_oreb = df_cut["OREB"].sum()
    league_dreb = df_cut["DREB"].sum()
    if league_oreb + league_dreb == 0:
        return math.nan
    league_avg = league_oreb / (league_oreb + league_dreb)

    # 6) combine
    return a_pct * (b_opp_pct / league_avg)

def get_second_chance_pts_per_100(team_name: str,
                                   season: str,
                                   date_str: str) -> float:
    """
    Returns team's second chance points per 100 possessions,
    where stats are up through (but not including) date_str.
    
    Formula: (PTS_2ND_CHANCE / POSS) * 100
    where POSS = PACE * MIN / 48
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"

    # 2) Get PTS_2ND_CHANCE from Misc stats
    df_misc = stats_getter.getLeagueDashTeamStats(
        team_name, season, date_from, cutoff,
        measure_type="Misc", per_mode="PerGame"
    )
    if df_misc.empty:
        return math.nan
    pts_2nd_chance = df_misc["PTS_2ND_CHANCE"].iloc[0]

    # 3) Get PACE and MIN from Base stats to calculate possessions
    df_advanced = stats_getter.getLeagueDashTeamStats(
        team_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if df_advanced.empty:
        return math.nan
    
    minutes = df_advanced["MIN"].iloc[0]
    poss = df_advanced['POSS'].iloc[0]
    gp = df_advanced['GP'].iloc[0]
    
    # Calculate possessions per game
   
    possessions_per_game = poss/gp
    
    if possessions_per_game == 0:
        return math.nan
    
    # 4) Calculate per 100 possessions
    pts_2nd_chance_per_100 = (pts_2nd_chance / possessions_per_game) * 100
    
    return pts_2nd_chance_per_100

def get_assist_ratio_relative(team_a_name: str,
                             team_b_name: str,
                             season: str,
                             date_str: str) -> float:
    """
    Returns A_astRatio * B_astRatio_defense,
    where B_astRatio_defense = (B_opp_astRatio / LeagueAvg_astRatio)
    
    - A_astRatio from Advanced stats (assists per 100 possessions)
    - B_opp_astRatio calculated from opponent assists and possessions
    - LeagueAvg_astRatio from previous season's league average
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"

    # 2) A_astRatio from Advanced
    dfA = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA.empty:
        return math.nan
    a_ast_ratio = dfA["AST_RATIO"].iloc[0]

    # 3) Get Team B's PACE and POSS from stats
    dfB_base = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Base", per_mode="PerGame"
    )
    if dfB_base.empty:
        return math.nan
    pace = dfB_base["PACE"].iloc[0]
    
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    b_poss = dfB_adv["POSS"].iloc[0]

    # 4) Get opponent assists from game log
    game_log = stats_getter.get_league_game_log(season).copy()
    game_log["GAME_DATE"] = pd.to_datetime(game_log["GAME_DATE"])
    
    # Filter games up to cutoff date
    game_log_filtered = game_log[game_log["GAME_DATE"] <= datetime.strptime(cutoff, "%m/%d/%Y")]
    
    # Get Team B's games as both home and away
    team_b_home = game_log_filtered[game_log_filtered["TEAM_NAME_HOME"] == team_b_name].copy()
    team_b_away = game_log_filtered[game_log_filtered["TEAM_NAME_AWAY"] == team_b_name].copy()
    
    # For home games, opponent assists are away team assists
    opp_ast_home = team_b_home["AST_AWAY"].sum() if "AST_AWAY" in team_b_home.columns else 0
    games_home = len(team_b_home)
    
    # For away games, opponent assists are home team assists  
    opp_ast_away = team_b_away["AST_HOME"].sum() if "AST_HOME" in team_b_away.columns else 0
    games_away = len(team_b_away)
    
    # Total opponent assists and games
    total_opp_ast = opp_ast_home + opp_ast_away
    total_games = games_home + games_away
    
    if total_games == 0:
        return math.nan
    
    # Average opponent assists per game
    opp_ast_per_game = total_opp_ast / total_games
    
    # 5) Calculate opponent possessions using pace formula
    # Pace = [240/(Team Minutes)] * (Possession_team + Possession_opponent)/2
    # With Team Minutes = 48: Pace = 5 * (b_poss + opp_poss)/2
    # Solving: opp_poss = (2 * Pace / 5) - b_poss
    opp_poss = (2 * pace / 5) - b_poss
    
    if opp_poss <= 0:
        return math.nan
    
    # 6) Calculate B_opp_astRatio (assists per 100 possessions)
    b_opp_ast_ratio = (opp_ast_per_game / opp_poss) * 100
    
    # 7) Get league average assist ratio from previous season
    league_avg_ast_ratio = get_league_average_stat("AST_RATIO", season)
    
    if league_avg_ast_ratio == 0:
        return math.nan
    
    # 8) Calculate B_astRatio_defense
    b_ast_ratio_defense = b_opp_ast_ratio / league_avg_ast_ratio
    
    # 9) Final calculation
    return a_ast_ratio * b_ast_ratio_defense

def get_turnover_ratio_relative(team_a_name: str,
                               team_b_name: str,
                               season: str,
                               date_str: str) -> float:
    """
    Returns (A_TOV_PCT * 100) * B_tovRatio_defense,
    where B_tovRatio_defense = (LeagueAvg_TOV_PCT * 100) / (B_opp_TOV_PCT * 100)
    
    - A_TOV_PCT from Advanced stats (turnovers per 100 possessions)
    - B_opp_TOV_PCT calculated from opponent turnovers and possessions
    - LeagueAvg_TOV_PCT from previous season's league average
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"

    # 2) A_TOV_PCT from Advanced
    dfA = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA.empty:
        return math.nan
    a_tov_pct = dfA["TM_TOV_PCT"].iloc[0]

    # 3) Get Team B's PACE and POSS from stats
    dfB_base = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Base", per_mode="PerGame"
    )
    if dfB_base.empty:
        return math.nan
    pace = dfB_base["PACE"].iloc[0]
    
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    b_poss = dfB_adv["POSS"].iloc[0]

    # 4) Get opponent turnovers from game log
    game_log = stats_getter.get_league_game_log(season).copy()
    game_log["GAME_DATE"] = pd.to_datetime(game_log["GAME_DATE"])
    
    # Filter games up to cutoff date
    game_log_filtered = game_log[game_log["GAME_DATE"] <= datetime.strptime(cutoff, "%m/%d/%Y")]
    
    # Get Team B's games as both home and away
    team_b_home = game_log_filtered[game_log_filtered["TEAM_NAME_HOME"] == team_b_name].copy()
    team_b_away = game_log_filtered[game_log_filtered["TEAM_NAME_AWAY"] == team_b_name].copy()
    
    # For home games, opponent turnovers are away team turnovers
    opp_tov_home = team_b_home["TOV_AWAY"].sum() if "TOV_AWAY" in team_b_home.columns else 0
    games_home = len(team_b_home)
    
    # For away games, opponent turnovers are home team turnovers  
    opp_tov_away = team_b_away["TOV_HOME"].sum() if "TOV_HOME" in team_b_away.columns else 0
    games_away = len(team_b_away)
    
    # Total opponent turnovers and games
    total_opp_tov = opp_tov_home + opp_tov_away
    total_games = games_home + games_away
    
    if total_games == 0:
        return math.nan
    
    # Average opponent turnovers per game
    opp_tov_per_game = total_opp_tov / total_games
    
    # 5) Calculate opponent possessions using pace formula
    # Pace = [240/(Team Minutes)] * (Possession_team + Possession_opponent)/2
    # With Team Minutes = 48: Pace = 5 * (b_poss + opp_poss)/2
    # Solving: opp_poss = (2 * Pace / 5) - b_poss
    opp_poss = (2 * pace / 5) - b_poss
    
    if opp_poss <= 0:
        return math.nan
    
    # 6) Calculate B_opp_TOV_PCT (turnovers per 100 possessions)
    b_opp_tov_pct = (opp_tov_per_game / opp_poss) * 100
    
    # 7) Get league average turnover percentage from previous season
    league_avg_tov_pct = get_league_average_stat("TOV_PCT", season)
    
    if b_opp_tov_pct == 0:
        return math.nan
    
    # 8) Calculate B_tovRatio_defense
    # Note: This is inverted because lower opponent turnovers = better defense
    b_tov_ratio_defense = (league_avg_tov_pct * 100) / (b_opp_tov_pct * 100)
    
    # 9) Final calculation
    return (a_tov_pct * 100) * b_tov_ratio_defense

def get_pct_ast_fgm_ratio(team_a_name: str,
                          team_b_name: str,
                          season: str,
                          date_str: str) -> float:
    """
    Returns A_PCT_AST_FGM / B_PCT_AST_FGM
    
    - A_PCT_AST_FGM: Percentage of Team A's field goals that are assisted
    - B_PCT_AST_FGM: Percentage of Team B's field goals that are assisted
    
    Both from Scoring stats
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"

    # 2) Get Team A's PCT_AST_FGM from Scoring stats
    dfA = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Scoring", per_mode="PerGame"
    )
    if dfA.empty:
        return math.nan
    a_pct_ast_fgm = dfA["PCT_AST_FGM"].iloc[0]

    # 3) Get Team B's PCT_AST_FGM from Scoring stats
    dfB = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Scoring", per_mode="PerGame"
    )
    if dfB.empty:
        return math.nan
    b_pct_ast_fgm = dfB["PCT_AST_FGM"].iloc[0]

    # 4) Calculate ratio
    if b_pct_ast_fgm == 0:
        return math.nan
    
    return a_pct_ast_fgm / b_pct_ast_fgm

def get_screen_assists_diff(team_a_name: str,
                           team_b_name: str,
                           season: str,
                           date_str: str) -> float:
    """
    Returns A_Screen_ast - B_Screen_ast
    
    Screen assists from Hustle stats showing how many times a screen led to a score
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"

    # 2) Get team IDs
    team_a_id = stats_getter.get_team_id(team_a_name)
    team_b_id = stats_getter.get_team_id(team_b_name)
    
    if not team_a_id or not team_b_id:
        return math.nan

    # 3) Get Team A's screen assists from Hustle stats
    dfA = stats_getter.getLeagueHustleTeamStats(
        team_id=team_a_id,
        season=season,
        date_from=date_from,
        date_to=cutoff,
        per_mode="PerGame"
    )
    if dfA.empty:
        return math.nan
    a_screen_ast = dfA["SCREEN_ASSISTS"].iloc[0]

    # 4) Get Team B's screen assists from Hustle stats
    dfB = stats_getter.getLeagueHustleTeamStats(
        team_id=team_b_id,
        season=season,
        date_from=date_from,
        date_to=cutoff,
        per_mode="PerGame"
    )
    if dfB.empty:
        return math.nan
    b_screen_ast = dfB["SCREEN_ASSISTS"].iloc[0]

    # 5) Calculate difference
    return a_screen_ast - b_screen_ast

def get_contested_3pt_rate_relative(team_a_name: str,
                                   team_b_name: str,
                                   season: str,
                                   date_str: str) -> float:
    """
    Returns Team A's contested 3PT rate relative to Team B's defense.
    
    Formula: ((A_veryTight_3FGA + A_tight_3FGA) / A_poss) 
             × ((B_opp_veryTight_3FGA + B_opp_tight_3FGA) / B_opp_poss) 
             × (A_veryTight_3FG% * B_Contested_3PT_Defense)
    
    Where B_Contested_3PT_Defense = (B_opp_contested3% / League Average Contested 3P%)
    """
    from nba_api.stats.endpoints import teamdashptshots
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    
    # 3) Get Team A's contested 3PT data
    dfs_a = get_team_pt_shots(team_a_name, season, date_from, cutoff, per_mode="PerGame")
    
    df_a = dfs_a[3]  # Team's own shooting
    a_very_tight = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '0-2 Feet - Very Tight']
    a_tight = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '2-4 Feet - Tight']
    
    a_very_tight_3fga = a_very_tight['FG3A'].iloc[0] if len(a_very_tight) > 0 else 0
    a_tight_3fga = a_tight['FG3A'].iloc[0] if len(a_tight) > 0 else 0
    
    # Get A's contested 3FG% (weighted average)
    a_very_tight_3fgm = a_very_tight['FG3M'].iloc[0] if len(a_very_tight) > 0 else 0
    a_tight_3fgm = a_tight['FG3M'].iloc[0] if len(a_tight) > 0 else 0
    
    total_contested_3fga = a_very_tight_3fga + a_tight_3fga
    total_contested_3fgm = a_very_tight_3fgm + a_tight_3fgm
    
    if total_contested_3fga == 0:
        a_contested_3fg_pct = 0
    else:
        a_contested_3fg_pct = total_contested_3fgm / total_contested_3fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 4) Get Team B's opponent contested 3PT data

    dfs_b = stats_getter.get_team_pt_shots(
        team_a_name, season, date_from, cutoff,
        per_mode="PerGame"
    )
    
    df_b_opp = dfs_b[4]  # Opponent shooting
    b_opp_very_tight = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '0-2 Feet - Very Tight']
    b_opp_tight = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '2-4 Feet - Tight']
    
    b_opp_very_tight_3fga = b_opp_very_tight['FG3A'].iloc[0] if len(b_opp_very_tight) > 0 else 0
    b_opp_tight_3fga = b_opp_tight['FG3A'].iloc[0] if len(b_opp_tight) > 0 else 0
    
    # Get B's opponent contested 3FG%
    b_opp_very_tight_3fgm = b_opp_very_tight['FG3M'].iloc[0] if len(b_opp_very_tight) > 0 else 0
    b_opp_tight_3fgm = b_opp_tight['FG3M'].iloc[0] if len(b_opp_tight) > 0 else 0
    
    b_opp_contested_3fga = b_opp_very_tight_3fga + b_opp_tight_3fga
    b_opp_contested_3fgm = b_opp_very_tight_3fgm + b_opp_tight_3fgm
    
    if b_opp_contested_3fga == 0:
        b_opp_contested_3fg_pct = 0
    else:
        b_opp_contested_3fg_pct = b_opp_contested_3fgm / b_opp_contested_3fga
    
    # Get B's opponent possessions (using pace formula)
   
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 5) Get league average contested 3P%
    league_avg_contested_3p_pct = get_league_average_stat("contested3_FG_PCT", season)
    
    # 6) Calculate components
    a_contested_rate = (a_very_tight_3fga + a_tight_3fga) / a_poss if a_poss > 0 else 0
    b_opp_contested_rate = (b_opp_very_tight_3fga + b_opp_tight_3fga) / b_opp_poss if b_opp_poss > 0 else 0
    
    if league_avg_contested_3p_pct == 0:
        b_contested_3pt_defense = 1
    else:
        b_contested_3pt_defense = b_opp_contested_3fg_pct / league_avg_contested_3p_pct
    
    # 7) Final calculation
    return a_contested_rate * b_opp_contested_rate * a_contested_3fg_pct * b_contested_3pt_defense

def get_open_3pt_rate_relative(team_a_name: str,
                               team_b_name: str,
                               season: str,
                               date_str: str) -> float:
    """
    Returns Team A's open 3PT rate relative to Team B's defense.
    
    Formula: ((A_open_3FGA + A_wideOpen_3FGA) / A_poss) 
             × ((B_opp_open_3FGA + B_opp_wideOpen_3FGA) / B_opp_poss) 
             × (A_open_3FG%)
    
    Where open = '4-6 Feet - Open' + '6+ Feet - Wide Open'
    """
    from nba_api.stats.endpoints import teamdashptshots
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
 
    # 3) Get Team A's open 3PT data
    dfs_a = stats_getter.get_team_pt_shots(
        team_a_name, season, date_from, cutoff,
        per_mode="PerGame"
    )
    
    df_a = dfs_a[3]  # Team's own shooting by defender distance
    a_open = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '4-6 Feet - Open']
    a_wide_open = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '6+ Feet - Wide Open']
    
    a_open_3fga = a_open['FG3A'].iloc[0] if len(a_open) > 0 else 0
    a_wide_open_3fga = a_wide_open['FG3A'].iloc[0] if len(a_wide_open) > 0 else 0
    
    # Get A's open 3FG% (weighted average)
    a_open_3fgm = a_open['FG3M'].iloc[0] if len(a_open) > 0 else 0
    a_wide_open_3fgm = a_wide_open['FG3M'].iloc[0] if len(a_wide_open) > 0 else 0
    
    total_open_3fga = a_open_3fga + a_wide_open_3fga
    total_open_3fgm = a_open_3fgm + a_wide_open_3fgm
    
    if total_open_3fga == 0:
        a_open_3fg_pct = 0
    else:
        a_open_3fg_pct = total_open_3fgm / total_open_3fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 4) Get Team B's opponent open 3PT data
    dfs_b = stats_getter.get_team_pt_shots(
        team_a_name, season, date_from, cutoff,
        per_mode="PerGame"
    )
    
    df_b_opp = dfs_b[4]  # Opponent shooting by defender distance
    b_opp_open = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '4-6 Feet - Open']
    b_opp_wide_open = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '6+ Feet - Wide Open']
    
    b_opp_open_3fga = b_opp_open['FG3A'].iloc[0] if len(b_opp_open) > 0 else 0
    b_opp_wide_open_3fga = b_opp_wide_open['FG3A'].iloc[0] if len(b_opp_wide_open) > 0 else 0
    
    # Get B's possessions and PACE from Advanced stats
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 5) Calculate components
    a_open_rate = (a_open_3fga + a_wide_open_3fga) / a_poss if a_poss > 0 else 0
    b_opp_open_rate = (b_opp_open_3fga + b_opp_wide_open_3fga) / b_opp_poss if b_opp_poss > 0 else 0
    
    # 6) Final calculation
    return a_open_rate * b_opp_open_rate * a_open_3fg_pct


def get_corner_3pt_rate_relative(team_a_name: str,
                                 team_b_name: str,
                                 season: str,
                                 date_str: str) -> float:
    """
    Returns Team A's corner 3PT rate relative to Team B's defense.
    
    Formula: (A_corner3_FGA / A_poss) × (B_opp_corner3_FGA / B_opp_poss) 
             × (A_corner3FG% * B_corner3_Defense)
    
    Where B_corner3_Defense = B_opp_corner3% / League Average corner 3P%
    """
    from nba_api.stats.endpoints import leaguedashteamshotlocations
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's corner 3 data
    df_a = get_team_shot_locations(team_a_name, season, date_from, cutoff)
    
    # Get corner 3 stats (combining left and right corners)
    a_left_corner = df_a[df_a['SHOT_CATEGORY'] == 'Left Corner 3']
    a_right_corner = df_a[df_a['SHOT_CATEGORY'] == 'Right Corner 3']
    
    a_left_fgm = a_left_corner['FGM'].iloc[0] if len(a_left_corner) > 0 else 0
    a_left_fga = a_left_corner['FGA'].iloc[0] if len(a_left_corner) > 0 else 0
    a_right_fgm = a_right_corner['FGM'].iloc[0] if len(a_right_corner) > 0 else 0
    a_right_fga = a_right_corner['FGA'].iloc[0] if len(a_right_corner) > 0 else 0
    
    a_corner3_fga = a_left_fga + a_right_fga
    a_corner3_fgm = a_left_fgm + a_right_fgm
    
    if a_corner3_fga == 0:
        a_corner3_fg_pct = 0
    else:
        a_corner3_fg_pct = a_corner3_fgm / a_corner3_fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 3) Get Team B's opponent corner 3 data
    df_b_opp = get_team_shot_locations(team_b_name, season, date_from, cutoff, measure_type="Opponent")
    
    # Get opponent corner 3 stats
    b_opp_left_corner = df_b_opp[df_b_opp['SHOT_CATEGORY'] == 'Left Corner 3']
    b_opp_right_corner = df_b_opp[df_b_opp['SHOT_CATEGORY'] == 'Right Corner 3']
    
    b_opp_left_fgm = b_opp_left_corner['OPP_FGM'].iloc[0] if len(b_opp_left_corner) > 0 else 0
    b_opp_left_fga = b_opp_left_corner['OPP_FGA'].iloc[0] if len(b_opp_left_corner) > 0 else 0
    b_opp_right_fgm = b_opp_right_corner['OPP_FGM'].iloc[0] if len(b_opp_right_corner) > 0 else 0
    b_opp_right_fga = b_opp_right_corner['OPP_FGA'].iloc[0] if len(b_opp_right_corner) > 0 else 0
    
    b_opp_corner3_fga = b_opp_left_fga + b_opp_right_fga
    b_opp_corner3_fgm = b_opp_left_fgm + b_opp_right_fgm
    
    if b_opp_corner3_fga == 0:
        b_opp_corner3_fg_pct = 0
    else:
        b_opp_corner3_fg_pct = b_opp_corner3_fgm / b_opp_corner3_fga
    
    # Get B's possessions and calculate opponent possessions
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 4) Get league average corner 3P%
    league_avg_corner3_pct = get_league_average_stat("corner3_FG_PCT", season)
    
    # 5) Calculate components
    a_corner3_rate = a_corner3_fga / a_poss if a_poss > 0 else 0
    b_opp_corner3_rate = b_opp_corner3_fga / b_opp_poss if b_opp_poss > 0 else 0
    
    if league_avg_corner3_pct == 0:
        b_corner3_defense = 1
    else:
        b_corner3_defense = b_opp_corner3_fg_pct / league_avg_corner3_pct
    
    # 6) Final calculation
    return a_corner3_rate * b_opp_corner3_rate * a_corner3_fg_pct * b_corner3_defense

def get_above_break_3pt_rate_relative(team_a_name: str,
                                      team_b_name: str,
                                      season: str,
                                      date_str: str) -> float:
    """
    Returns Team A's above-the-break 3PT rate relative to Team B's defense.
    
    Formula: (A_aboveBreak3_FGA / A_poss) × (B_opp_aboveBreak3_FGA / B_opp_poss) 
             × (A_aboveBreak3_FG% * B_aboveBreak3_Defense)
    
    Where B_aboveBreak3_Defense = B_opp_aboveBreak3% / League Average aboveBreak 3P%
    """
    from nba_api.stats.endpoints import leaguedashteamshotlocations
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's above-the-break 3 data
    df_a = get_team_shot_locations(team_a_name, season, date_from, cutoff)
    
    # Get above-the-break 3 stats
    a_above_break = df_a[df_a['SHOT_CATEGORY'] == 'Above the Break 3']
    
    if len(a_above_break) > 0:
        a_above_break_fgm = a_above_break['FGM'].iloc[0]
        a_above_break_fga = a_above_break['FGA'].iloc[0]
    else:
        a_above_break_fgm = 0
        a_above_break_fga = 0
    
    if a_above_break_fga == 0:
        a_above_break_fg_pct = 0
    else:
        a_above_break_fg_pct = a_above_break_fgm / a_above_break_fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 3) Get Team B's opponent above-the-break 3 data
    df_b_opp = get_team_shot_locations(team_b_name, season, date_from, cutoff, measure_type="Opponent")
    
    # Get opponent above-the-break 3 stats
    b_opp_above_break = df_b_opp[df_b_opp['SHOT_CATEGORY'] == 'Above the Break 3']
    
    if len(b_opp_above_break) > 0:
        b_opp_above_break_fgm = b_opp_above_break['OPP_FGM'].iloc[0]
        b_opp_above_break_fga = b_opp_above_break['OPP_FGA'].iloc[0]
    else:
        b_opp_above_break_fgm = 0
        b_opp_above_break_fga = 0
    
    if b_opp_above_break_fga == 0:
        b_opp_above_break_fg_pct = 0
    else:
        b_opp_above_break_fg_pct = b_opp_above_break_fgm / b_opp_above_break_fga
    
    # Get B's possessions and calculate opponent possessions
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 4) Get league average above-the-break 3P%
    league_avg_above_break_pct = get_league_average_stat("aboveBreak3_FG_PCT", season)
    
    # 5) Calculate components
    a_above_break_rate = a_above_break_fga / a_poss if a_poss > 0 else 0
    b_opp_above_break_rate = b_opp_above_break_fga / b_opp_poss if b_opp_poss > 0 else 0
    
    if league_avg_above_break_pct == 0:
        b_above_break_defense = 1
    else:
        b_above_break_defense = b_opp_above_break_fg_pct / league_avg_above_break_pct
    
    # 6) Final calculation
    return a_above_break_rate * b_opp_above_break_rate * a_above_break_fg_pct * b_above_break_defense

def get_paint_shot_rate_relative(team_a_name: str,
                                 team_b_name: str,
                                 season: str,
                                 date_str: str) -> float:
    """
    Returns Team A's paint shot rate (including restricted area) relative to Team B's defense.
    
    Formula: ((A_paint_FGA + A_restrictedArea_FGA) / A_poss) 
             × ((B_opp_paint_FGA + B_opp_restrictedArea_FGA) / B_opp_poss)
             × (A_paintFG% * B_paint_Defense)
    
    Where B_paint_Defense = B_opp_paintFG% / LeagueAvg_PaintFG%
    Paint shots include 'In The Paint (Non-RA)' and 'Restricted Area'
    """
    from nba_api.stats.endpoints import leaguedashteamshotlocations
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's paint data
    df_a = get_team_shot_locations(team_a_name, season, date_from, cutoff)
   
    # Get paint and restricted area stats
    a_paint = df_a[df_a['SHOT_CATEGORY'] == 'In The Paint (Non-RA)']
    a_restricted = df_a[df_a['SHOT_CATEGORY'] == 'Restricted Area']
    
    a_paint_fgm = a_paint['FGM'].iloc[0] if len(a_paint) > 0 else 0
    a_paint_fga = a_paint['FGA'].iloc[0] if len(a_paint) > 0 else 0
    a_restricted_fgm = a_restricted['FGM'].iloc[0] if len(a_restricted) > 0 else 0
    a_restricted_fga = a_restricted['FGA'].iloc[0] if len(a_restricted) > 0 else 0
    
    # Combine paint and restricted area
    a_total_paint_fga = a_paint_fga + a_restricted_fga
    a_total_paint_fgm = a_paint_fgm + a_restricted_fgm
    
    if a_total_paint_fga == 0:
        a_paint_fg_pct = 0
    else:
        a_paint_fg_pct = a_total_paint_fgm / a_total_paint_fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 3) Get Team B's opponent paint data
    df_b_opp = get_team_shot_locations(team_b_name, season, date_from, cutoff, measure_type="Opponent")
    
    # Get opponent paint and restricted area stats
    b_opp_paint = df_b_opp[df_b_opp['SHOT_CATEGORY'] == 'In The Paint (Non-RA)']
    b_opp_restricted = df_b_opp[df_b_opp['SHOT_CATEGORY'] == 'Restricted Area']
    
    b_opp_paint_fgm = b_opp_paint['OPP_FGM'].iloc[0] if len(b_opp_paint) > 0 else 0
    b_opp_paint_fga = b_opp_paint['OPP_FGA'].iloc[0] if len(b_opp_paint) > 0 else 0
    b_opp_restricted_fgm = b_opp_restricted['OPP_FGM'].iloc[0] if len(b_opp_restricted) > 0 else 0
    b_opp_restricted_fga = b_opp_restricted['OPP_FGA'].iloc[0] if len(b_opp_restricted) > 0 else 0
    
    # Combine opponent paint and restricted area
    b_opp_total_paint_fga = b_opp_paint_fga + b_opp_restricted_fga
    b_opp_total_paint_fgm = b_opp_paint_fgm + b_opp_restricted_fgm
    
    if b_opp_total_paint_fga == 0:
        b_opp_paint_fg_pct = 0
    else:
        b_opp_paint_fg_pct = b_opp_total_paint_fgm / b_opp_total_paint_fga
    
    # Get B's possessions and calculate opponent possessions
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 4) Get league average paint FG% (including restricted area)
    league_avg_paint_pct = get_league_average_stat("paint_FG_PCT", season)
    
    # 5) Calculate components
    a_paint_rate = a_total_paint_fga / a_poss if a_poss > 0 else 0
    b_opp_paint_rate = b_opp_total_paint_fga / b_opp_poss if b_opp_poss > 0 else 0
    
    if league_avg_paint_pct == 0:
        b_paint_defense = 1
    else:
        b_paint_defense = b_opp_paint_fg_pct / league_avg_paint_pct
    
    # 6) Final calculation
    return a_paint_rate * b_opp_paint_rate * a_paint_fg_pct * b_paint_defense

def get_midrange_shot_rate_relative(team_a_name: str,
                                    team_b_name: str,
                                    season: str,
                                    date_str: str) -> float:
    """
    Returns Team A's mid-range shot rate relative to Team B's defense.
    
    Formula: (A_midRange_FGA / A_poss) × (B_opp_midRange_FGA / B_opp_poss)
             × (A_midRangeFG% * B_midRange_Defense)
    
    Where B_midRange_Defense = B_opp_midRangeFG% / LeagueAverage_midRangeFG%
    """
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's mid-range data
    df_a = get_team_shot_locations(team_a_name, season, date_from, cutoff)
    
    # Get mid-range stats
    a_midrange = df_a[df_a['SHOT_CATEGORY'] == 'Mid-Range']
    
    if len(a_midrange) > 0:
        a_midrange_fgm = a_midrange['FGM'].iloc[0]
        a_midrange_fga = a_midrange['FGA'].iloc[0]
    else:
        a_midrange_fgm = 0
        a_midrange_fga = 0
    
    if a_midrange_fga == 0:
        a_midrange_fg_pct = 0
    else:
        a_midrange_fg_pct = a_midrange_fgm / a_midrange_fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 3) Get Team B's opponent mid-range data
    df_b_opp = get_team_shot_locations(team_b_name, season, date_from, cutoff, measure_type="Opponent")
    
    # Get opponent mid-range stats
    b_opp_midrange = df_b_opp[df_b_opp['SHOT_CATEGORY'] == 'Mid-Range']
    
    if len(b_opp_midrange) > 0:
        b_opp_midrange_fgm = b_opp_midrange['OPP_FGM'].iloc[0]
        b_opp_midrange_fga = b_opp_midrange['OPP_FGA'].iloc[0]
    else:
        b_opp_midrange_fgm = 0
        b_opp_midrange_fga = 0
    
    if b_opp_midrange_fga == 0:
        b_opp_midrange_fg_pct = 0
    else:
        b_opp_midrange_fg_pct = b_opp_midrange_fgm / b_opp_midrange_fga
    
    # Get B's possessions and calculate opponent possessions
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 4) Get league average mid-range FG%
    league_avg_midrange_pct = get_league_average_stat("midRange_FG_PCT", season)
    
    # 5) Calculate components
    a_midrange_rate = a_midrange_fga / a_poss if a_poss > 0 else 0
    b_opp_midrange_rate = b_opp_midrange_fga / b_opp_poss if b_opp_poss > 0 else 0
    
    if league_avg_midrange_pct == 0:
        b_midrange_defense = 1
    else:
        b_midrange_defense = b_opp_midrange_fg_pct / league_avg_midrange_pct
    
    # 6) Final calculation
    return a_midrange_rate * b_opp_midrange_rate * a_midrange_fg_pct * b_midrange_defense

def get_contested_2pt_rate_relative(team_a_name: str,
                                   team_b_name: str,
                                   season: str,
                                   date_str: str) -> float:
    """
    Returns Team A's contested 2PT rate relative to Team B's defense.
    
    Formula: ((A_veryTight_2FGA + A_tight_2FGA) / A_poss) 
             × ((B_opp_veryTight_2FGA + B_opp_tight_2FGA) / B_opp_poss) 
             × (A_contested_2FG% * B_Contested_2PT_Defense)
    
    Where B_Contested_2PT_Defense = (B_opp_contested2% / League Average Contested 2P%)
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's contested 2PT data
    dfs_a = get_team_pt_shots(team_a_name, season, date_from, cutoff, per_mode="PerGame")
    
    df_a = dfs_a[3]  # Team's own shooting
    a_very_tight = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '0-2 Feet - Very Tight']
    a_tight = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '2-4 Feet - Tight']
    
    # Get 2PT attempts (FGA - FG3A = 2PT attempts)
    a_very_tight_2fga = 0
    a_tight_2fga = 0
    a_very_tight_2fgm = 0
    a_tight_2fgm = 0
    
    if len(a_very_tight) > 0:
        a_very_tight_2fga = a_very_tight['FGA'].iloc[0] - a_very_tight['FG3A'].iloc[0]
        a_very_tight_2fgm = a_very_tight['FGM'].iloc[0] - a_very_tight['FG3M'].iloc[0]
    
    if len(a_tight) > 0:
        a_tight_2fga = a_tight['FGA'].iloc[0] - a_tight['FG3A'].iloc[0]
        a_tight_2fgm = a_tight['FGM'].iloc[0] - a_tight['FG3M'].iloc[0]
    
    # Get A's contested 2FG% (weighted average)
    total_contested_2fga = a_very_tight_2fga + a_tight_2fga
    total_contested_2fgm = a_very_tight_2fgm + a_tight_2fgm
    
    if total_contested_2fga == 0:
        a_contested_2fg_pct = 0
    else:
        a_contested_2fg_pct = total_contested_2fgm / total_contested_2fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 3) Get Team B's opponent contested 2PT data
    dfs_b = stats_getter.get_team_pt_shots(
        team_b_name, season, date_from, cutoff,
        per_mode="PerGame"
    )
    
    df_b_opp = dfs_b[4]  # Opponent shooting
    b_opp_very_tight = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '0-2 Feet - Very Tight']
    b_opp_tight = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '2-4 Feet - Tight']
    
    # Get opponent 2PT attempts
    b_opp_very_tight_2fga = 0
    b_opp_tight_2fga = 0
    b_opp_very_tight_2fgm = 0
    b_opp_tight_2fgm = 0
    
    if len(b_opp_very_tight) > 0:
        b_opp_very_tight_2fga = b_opp_very_tight['FGA'].iloc[0] - b_opp_very_tight['FG3A'].iloc[0]
        b_opp_very_tight_2fgm = b_opp_very_tight['FGM'].iloc[0] - b_opp_very_tight['FG3M'].iloc[0]
    
    if len(b_opp_tight) > 0:
        b_opp_tight_2fga = b_opp_tight['FGA'].iloc[0] - b_opp_tight['FG3A'].iloc[0]
        b_opp_tight_2fgm = b_opp_tight['FGM'].iloc[0] - b_opp_tight['FG3M'].iloc[0]
    
    # Get B's opponent contested 2FG%
    b_opp_contested_2fga = b_opp_very_tight_2fga + b_opp_tight_2fga
    b_opp_contested_2fgm = b_opp_very_tight_2fgm + b_opp_tight_2fgm
    
    if b_opp_contested_2fga == 0:
        b_opp_contested_2fg_pct = 0
    else:
        b_opp_contested_2fg_pct = b_opp_contested_2fgm / b_opp_contested_2fga
    
    # Get B's opponent possessions (using pace formula)
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 4) Get league average contested 2P%
    league_avg_contested_2p_pct = get_league_average_stat("contested2_FG_PCT", season)
    
    # 5) Calculate components
    a_contested_rate = (a_very_tight_2fga + a_tight_2fga) / a_poss if a_poss > 0 else 0
    b_opp_contested_rate = (b_opp_very_tight_2fga + b_opp_tight_2fga) / b_opp_poss if b_opp_poss > 0 else 0
    
    if league_avg_contested_2p_pct == 0:
        b_contested_2pt_defense = 1
    else:
        b_contested_2pt_defense = b_opp_contested_2fg_pct / league_avg_contested_2p_pct
    
    # 6) Final calculation
    return a_contested_rate * b_opp_contested_rate * a_contested_2fg_pct * b_contested_2pt_defense

def get_contested_2pt_rate_relative(team_a_name: str,
                                   team_b_name: str,
                                   season: str,
                                   date_str: str) -> float:
    """
    Returns Team A's contested 2PT rate relative to Team B's defense.
    
    Formula: ((A_veryTight_2FGA + A_tight_2FGA) / A_poss) 
             × ((B_opp_veryTight_2FGA + B_opp_tight_2FGA) / B_opp_poss) 
             × (A_contested_2FG% * B_Contested_2PT_Defense)
    
    Where B_Contested_2PT_Defense = (B_opp_contested2% / League Average Contested 2P%)
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's contested 2PT data
    dfs_a = get_team_pt_shots(team_a_name, season, date_from, cutoff, per_mode="PerGame")
    
    df_a = dfs_a[3]  # Team's own shooting
    a_very_tight = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '0-2 Feet - Very Tight']
    a_tight = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '2-4 Feet - Tight']
    
    # Get 2PT attempts (FGA - FG3A = 2PT attempts)
    a_very_tight_2fga = 0
    a_tight_2fga = 0
    a_very_tight_2fgm = 0
    a_tight_2fgm = 0
    
    if len(a_very_tight) > 0:
        a_very_tight_2fga = a_very_tight['FGA'].iloc[0] - a_very_tight['FG3A'].iloc[0]
        a_very_tight_2fgm = a_very_tight['FGM'].iloc[0] - a_very_tight['FG3M'].iloc[0]
    
    if len(a_tight) > 0:
        a_tight_2fga = a_tight['FGA'].iloc[0] - a_tight['FG3A'].iloc[0]
        a_tight_2fgm = a_tight['FGM'].iloc[0] - a_tight['FG3M'].iloc[0]
    
    # Get A's contested 2FG% (weighted average)
    total_contested_2fga = a_very_tight_2fga + a_tight_2fga
    total_contested_2fgm = a_very_tight_2fgm + a_tight_2fgm
    
    if total_contested_2fga == 0:
        a_contested_2fg_pct = 0
    else:
        a_contested_2fg_pct = total_contested_2fgm / total_contested_2fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 3) Get Team B's opponent contested 2PT data
    dfs_b = stats_getter.get_team_pt_shots(
        team_b_name, season, date_from, cutoff,
        per_mode="PerGame"
    )
    
    df_b_opp = dfs_b[4]  # Opponent shooting
    b_opp_very_tight = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '0-2 Feet - Very Tight']
    b_opp_tight = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '2-4 Feet - Tight']
    
    # Get opponent 2PT attempts
    b_opp_very_tight_2fga = 0
    b_opp_tight_2fga = 0
    b_opp_very_tight_2fgm = 0
    b_opp_tight_2fgm = 0
    
    if len(b_opp_very_tight) > 0:
        b_opp_very_tight_2fga = b_opp_very_tight['FGA'].iloc[0] - b_opp_very_tight['FG3A'].iloc[0]
        b_opp_very_tight_2fgm = b_opp_very_tight['FGM'].iloc[0] - b_opp_very_tight['FG3M'].iloc[0]
    
    if len(b_opp_tight) > 0:
        b_opp_tight_2fga = b_opp_tight['FGA'].iloc[0] - b_opp_tight['FG3A'].iloc[0]
        b_opp_tight_2fgm = b_opp_tight['FGM'].iloc[0] - b_opp_tight['FG3M'].iloc[0]
    
    # Get B's opponent contested 2FG%
    b_opp_contested_2fga = b_opp_very_tight_2fga + b_opp_tight_2fga
    b_opp_contested_2fgm = b_opp_very_tight_2fgm + b_opp_tight_2fgm
    
    if b_opp_contested_2fga == 0:
        b_opp_contested_2fg_pct = 0
    else:
        b_opp_contested_2fg_pct = b_opp_contested_2fgm / b_opp_contested_2fga
    
    # Get B's opponent possessions (using pace formula)
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 4) Get league average contested 2P%
    league_avg_contested_2p_pct = get_league_average_stat("contested2_FG_PCT", season)
    
    # 5) Calculate components
    a_contested_rate = (a_very_tight_2fga + a_tight_2fga) / a_poss if a_poss > 0 else 0
    b_opp_contested_rate = (b_opp_very_tight_2fga + b_opp_tight_2fga) / b_opp_poss if b_opp_poss > 0 else 0
    
    if league_avg_contested_2p_pct == 0:
        b_contested_2pt_defense = 1
    else:
        b_contested_2pt_defense = b_opp_contested_2fg_pct / league_avg_contested_2p_pct
    
    # 6) Final calculation
    return a_contested_rate * b_opp_contested_rate * a_contested_2fg_pct * b_contested_2pt_defense


def get_open_2pt_rate_relative(team_a_name: str,
                               team_b_name: str,
                               season: str,
                               date_str: str) -> float:
    """
    Returns Team A's open 2PT rate relative to Team B's defense.
    
    Formula: ((A_open_2FGA + A_wideOpen_2FGA) / A_poss) 
             × ((B_opp_open_2FGA + B_opp_wideOpen_2FGA) / B_opp_poss) 
             × (A_open2FG%)
    
    Where open = '4-6 Feet - Open' + '6+ Feet - Wide Open'
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's open 2PT data
    dfs_a = stats_getter.get_team_pt_shots(
        team_a_name, season, date_from, cutoff,
        per_mode="PerGame"
    )
    
    df_a = dfs_a[3]  # Team's own shooting by defender distance
    a_open = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '4-6 Feet - Open']
    a_wide_open = df_a[df_a['CLOSE_DEF_DIST_RANGE'] == '6+ Feet - Wide Open']
    
    # Get 2PT attempts (FGA - FG3A = 2PT attempts)
    a_open_2fga = 0
    a_wide_open_2fga = 0
    a_open_2fgm = 0
    a_wide_open_2fgm = 0
    
    if len(a_open) > 0:
        a_open_2fga = a_open['FGA'].iloc[0] - a_open['FG3A'].iloc[0]
        a_open_2fgm = a_open['FGM'].iloc[0] - a_open['FG3M'].iloc[0]
    
    if len(a_wide_open) > 0:
        a_wide_open_2fga = a_wide_open['FGA'].iloc[0] - a_wide_open['FG3A'].iloc[0]
        a_wide_open_2fgm = a_wide_open['FGM'].iloc[0] - a_wide_open['FG3M'].iloc[0]
    
    # Get A's open 2FG% (weighted average)
    total_open_2fga = a_open_2fga + a_wide_open_2fga
    total_open_2fgm = a_open_2fgm + a_wide_open_2fgm
    
    if total_open_2fga == 0:
        a_open_2fg_pct = 0
    else:
        a_open_2fg_pct = total_open_2fgm / total_open_2fga
    
    # Get Team A's possessions
    dfA_adv = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_adv.empty:
        return math.nan
    a_poss = dfA_adv["POSS"].iloc[0]
    
    # 3) Get Team B's opponent open 2PT data
    dfs_b = stats_getter.get_team_pt_shots(
        team_b_name, season, date_from, cutoff,
        per_mode="PerGame"
    )
    
    df_b_opp = dfs_b[4]  # Opponent shooting by defender distance
    b_opp_open = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '4-6 Feet - Open']
    b_opp_wide_open = df_b_opp[df_b_opp['CLOSE_DEF_DIST_RANGE'] == '6+ Feet - Wide Open']
    
    # Get opponent 2PT attempts
    b_opp_open_2fga = 0
    b_opp_wide_open_2fga = 0
    
    if len(b_opp_open) > 0:
        b_opp_open_2fga = b_opp_open['FGA'].iloc[0] - b_opp_open['FG3A'].iloc[0]
    
    if len(b_opp_wide_open) > 0:
        b_opp_wide_open_2fga = b_opp_wide_open['FGA'].iloc[0] - b_opp_wide_open['FG3A'].iloc[0]
    
    # Get B's possessions and PACE from Advanced stats
    dfB_adv = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_adv.empty:
        return math.nan
    
    pace = dfB_adv["PACE"].iloc[0]
    b_poss = dfB_adv["POSS"].iloc[0]
    b_opp_poss = (2 * pace / 5) - b_poss
    
    # 4) Calculate components
    a_open_rate = (a_open_2fga + a_wide_open_2fga) / a_poss if a_poss > 0 else 0
    b_opp_open_rate = (b_opp_open_2fga + b_opp_wide_open_2fga) / b_opp_poss if b_opp_poss > 0 else 0
    
    # 5) Final calculation
    return a_open_rate * b_opp_open_rate * a_open_2fg_pct

def get_pts_off_tov_rate_relative(team_a_name: str,
                                 team_b_name: str,
                                 season: str,
                                 date_str: str) -> float:
    """
    Returns Team A's PTS off TOV rate relative to Team B's defense.
    
    Formula: A_PTS_OFF_TOV × (B_opp_PTS_OFF_TOV / LeagueAvg_PTS_OFF_TOV)
    
    Where:
    - A_PTS_OFF_TOV = Team A's points off turnovers per game
    - B_opp_PTS_OFF_TOV = Average PTS off TOV that Team B's opponents score per game
    - LeagueAvg_PTS_OFF_TOV = League average points off turnovers per game
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's PTS_OFF_TOV data
    dfA_misc = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Misc", per_mode="PerGame"
    )
    if dfA_misc.empty:
        return math.nan
    
    a_pts_off_tov = dfA_misc["PTS_OFF_TOV"].iloc[0]
    
    # 3) Get Team B's opponent PTS_OFF_TOV data
    dfB_misc = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Misc", per_mode="PerGame"
    )
    if dfB_misc.empty:
        return math.nan
    
    b_opp_pts_off_tov = dfB_misc["OPP_PTS_OFF_TOV"].iloc[0]
    
    # 4) Get league average PTS_OFF_TOV
    league_avg_pts_off_tov = get_league_average_stat("PTS_OFF_TOV", season)
    
    # 5) Calculate the relative rate
    if league_avg_pts_off_tov == 0:
        defensive_factor = 1
    else:
        defensive_factor = b_opp_pts_off_tov / league_avg_pts_off_tov
    
    # 6) Final calculation
    return a_pts_off_tov * defensive_factor

def get_pts_fastbreak_rate_relative(team_a_name: str, team_b_name: str, 
                                   season: str, date_str: str) -> float:
    """
    Similar to get_pts_off_tov_rate_relative but for fast break points.
    A_PTS_FB × (B_opp_PTS_FB / LeagueAvg_PTS_FB)
    """
    # 1) build date window
    target    = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff    = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr  = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # 2) Get Team A's PTS_OFF_TOV data
    dfA_misc = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, date_from, cutoff,
        measure_type="Misc", per_mode="PerGame"
    )
    if dfA_misc.empty:
        return math.nan
    
    a_pts_fb = dfA_misc["PTS_FB"].iloc[0]
    
    # 3) Get Team B's opponent PTS_OFF_TOV data
    dfB_misc = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, date_from, cutoff,
        measure_type="Misc", per_mode="PerGame"
    )
    if dfB_misc.empty:
        return math.nan
    
    b_opp_pts_fb = dfB_misc["OPP_PTS_FB"].iloc[0]
    
    # 4) Get league average PTS_OFF_TOV
    league_avg_pts_fb = get_league_average_stat("PTS_FB", season)

    return (a_pts_fb * (b_opp_pts_fb / league_avg_pts_fb))
   



def get_team_last_5_games_date(team_name: str, target_date: str, season: str) -> str:
        """
        Find the date that corresponds to 5 games ago for a team.
        """
        # Get full season game log for the team
        df_all = stats_getter.get_league_game_log(season)
        
        # Filter for the specific team
        team_id = get_team_id(team_name)
        if not team_id:
            return None
            
        team_games = df_all[df_all['TEAM_ID'] == team_id].copy()
        
        # Convert dates and filter games before target date
        team_games['GAME_DATE'] = pd.to_datetime(team_games['GAME_DATE'])
        target_dt = datetime.strptime(target_date, "%m/%d/%Y")
        
        games_before = team_games[team_games['GAME_DATE'] < target_dt]
        
        if len(games_before) < 5:
            # Not enough games, use season start
            start_yr = int(season.split("-")[0])
            return f"10/01/{start_yr}"
        
        # Sort by date descending and get the 5th most recent game
        games_before = games_before.sort_values('GAME_DATE', ascending=False)
        fifth_game_date = games_before.iloc[4]['GAME_DATE']  # 0-indexed, so 4 = 5th game
        
        return fifth_game_date.strftime("%m/%d/%Y")

def get_recent_netrtg_ratio(team_a_name: str,
                            team_b_name: str,
                            season: str,
                            date_str: str) -> float:
    """
    Returns the ratio of Team A's NETRTG over last 5 games to Team B's NETRTG over last 5 games.
    
    Formula: A_NETRTG_last5 / B_NETRTG_last5
    
    Where NETRTG_last5 is calculated from the "Four Factors" endpoint over the team's 
    last 5 games prior to the given date.
    """
       # 1) Get the date for last 5 games for each team
    team_a_last5_date = get_team_last_5_games_date(team_a_name, date_str, season)
    team_b_last5_date = get_team_last_5_games_date(team_b_name, date_str, season)
    
    if not team_a_last5_date or not team_b_last5_date:
        return math.nan
    
    # 2) Calculate cutoff (day before target)
    target = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    
    # 3) Get Team A's NETRTG over last 5 games
    dfA_ff = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, team_a_last5_date, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfA_ff.empty:
        return math.nan
    
    a_netrtg_last5 = dfA_ff["NET_RATING"].iloc[0]
    
    # 4) Get Team B's NETRTG over last 5 games  
    dfB_ff = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, team_b_last5_date, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    if dfB_ff.empty:
        return math.nan
    
    b_netrtg_last5 = dfB_ff["NET_RATING"].iloc[0]
    
    # 5) Calculate ratio
    if b_netrtg_last5 == 0:
        return math.nan  # Avoid division by zero
    
    return a_netrtg_last5 / b_netrtg_last5


def get_recent_oreb_pct_ratio(team_a_name: str,
                              team_b_name: str,
                              season: str,
                              date_str: str) -> float:
    """
    Returns the ratio of Team A's OREB% over last 5 games to Team B's OREB% over last 5 games.
    
    Formula: A_OREB_PCT_LAST5 / B_OREB_PCT_LAST5
    
    Where OREB_PCT_LAST5 is calculated from the "Four Factors" endpoint over the team's 
    last 5 games prior to the given date.
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) Get the date for last 5 games for each team
    team_a_last5_date = get_team_last_5_games_date(team_a_name, date_str, season)
    team_b_last5_date = get_team_last_5_games_date(team_b_name, date_str, season)
    
    if not team_a_last5_date or not team_b_last5_date:
        return math.nan
    
    # 2) Calculate cutoff (day before target)
    target = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    
    # 3) Get Team A's OREB% over last 5 games
    dfA_ff = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, team_a_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfA_ff.empty:
        return math.nan
    
    a_oreb_pct_last5 = dfA_ff["OREB_PCT"].iloc[0]
    
    # 4) Get Team B's OREB% over last 5 games  
    dfB_ff = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, team_b_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfB_ff.empty:
        return math.nan
    
    b_oreb_pct_last5 = dfB_ff["OREB_PCT"].iloc[0]
    
    # 5) Calculate ratio
    if b_oreb_pct_last5 == 0:
        return math.nan  # Avoid division by zero
    
    return a_oreb_pct_last5 / b_oreb_pct_last5

def get_recent_efg_pct_ratio(team_a_name: str,
                            team_b_name: str,
                            season: str,
                            date_str: str) -> float:
    """
    Returns the ratio of Team A's eFG% over last 5 games to Team B's eFG% over last 5 games.
    
    Formula: A_eFG%_LAST5 / B_eFG%_LAST5
    
    Where eFG%_LAST5 is calculated from the "Four Factors" endpoint over the team's 
    last 5 games prior to the given date.
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) Get the date for last 5 games for each team
    team_a_last5_date = get_team_last_5_games_date(team_a_name, date_str, season)
    team_b_last5_date = get_team_last_5_games_date(team_b_name, date_str, season)
    
    if not team_a_last5_date or not team_b_last5_date:
        return math.nan
    
    # 2) Calculate cutoff (day before target)
    target = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    
    # 3) Get Team A's eFG% over last 5 games
    dfA_ff = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, team_a_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfA_ff.empty:
        return math.nan
    
    a_efg_pct_last5 = dfA_ff["EFG_PCT"].iloc[0]
    
    # 4) Get Team B's eFG% over last 5 games  
    dfB_ff = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, team_b_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfB_ff.empty:
        return math.nan
    
    b_efg_pct_last5 = dfB_ff["EFG_PCT"].iloc[0]
    
    # 5) Calculate ratio
    if b_efg_pct_last5 == 0:
        return math.nan  # Avoid division by zero
    
    return a_efg_pct_last5 / b_efg_pct_last5      

def get_recent_tov_pct_ratio(team_a_name: str,
                             team_b_name: str,
                             season: str,
                             date_str: str) -> float:
    """
    Returns the ratio of Team A's TOV% over last 5 games to Team B's TOV% over last 5 games.
    
    Formula: A_TOV% / B_TOV%
    
    Where TOV% is calculated from the "Four Factors" endpoint over the team's 
    last 5 games prior to the given date.
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) Get the date for last 5 games for each team
    team_a_last5_date = get_team_last_5_games_date(team_a_name, date_str, season)
    team_b_last5_date = get_team_last_5_games_date(team_b_name, date_str, season)
    
    if not team_a_last5_date or not team_b_last5_date:
        return math.nan
    
    # 2) Calculate cutoff (day before target)
    target = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    
    # 3) Get Team A's TOV% over last 5 games
    dfA_ff = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, team_a_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfA_ff.empty:
        return math.nan
    
    a_tov_pct_last5 = dfA_ff["TM_TOV_PCT"].iloc[0]
    
    # 4) Get Team B's TOV% over last 5 games  
    dfB_ff = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, team_b_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfB_ff.empty:
        return math.nan
    
    b_tov_pct_last5 = dfB_ff["TM_TOV_PCT"].iloc[0]
    
    # 5) Calculate ratio
    if b_tov_pct_last5 == 0:
        return math.nan  # Avoid division by zero
    
    return a_tov_pct_last5 / b_tov_pct_last5


def get_recent_ft_rate_ratio(team_a_name: str,
                            team_b_name: str,
                            season: str,
                            date_str: str) -> float:
    """
    Returns the ratio of Team A's FT Rate over last 5 games to Team B's FT Rate over last 5 games.
    
    Formula: A_FTA_RATE / B_FTA_RATE
    
    Where FTA_RATE is calculated from the "Four Factors" endpoint over the team's 
    last 5 games prior to the given date.
    """
    import math
    from datetime import datetime, timedelta
    
    # 1) Get the date for last 5 games for each team
    team_a_last5_date = get_team_last_5_games_date(team_a_name, date_str, season)
    team_b_last5_date = get_team_last_5_games_date(team_b_name, date_str, season)
    
    if not team_a_last5_date or not team_b_last5_date:
        return math.nan
    
    # 2) Calculate cutoff (day before target)
    target = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    
    # 3) Get Team A's FT Rate over last 5 games
    dfA_ff = stats_getter.getLeagueDashTeamStats(
        team_a_name, season, team_a_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfA_ff.empty:
        return math.nan
    
    a_ft_rate_last5 = dfA_ff["FTA_RATE"].iloc[0]
    
    # 4) Get Team B's FT Rate over last 5 games  
    dfB_ff = stats_getter.getLeagueDashTeamStats(
        team_b_name, season, team_b_last5_date, cutoff,
        measure_type="Four Factors", per_mode="PerGame"
    )
    if dfB_ff.empty:
        return math.nan
    
    b_ft_rate_last5 = dfB_ff["FTA_RATE"].iloc[0]
    
    # 5) Calculate ratio
    if b_ft_rate_last5 == 0:
        return math.nan  # Avoid division by zero
    
    return a_ft_rate_last5 / b_ft_rate_last5

def get_last_season_NETRTG(team_name: str, season: str) -> float:
    """
    Returns the team's average NET_RATING for the previous season.
    
    Parameters:
        team_name (str): Full name of the team (e.g., 'Los Angeles Lakers')
        season (str): Current season in format "YYYY-YY" (e.g., "2019-20")
    
    Returns:
        float: Team's NET_RATING for the previous season, or math.nan if no data found
    """
    # 1) Calculate the previous season
    current_start_year = int(season.split("-")[0])
    prev_start_year = current_start_year - 1
    prev_season = f"{prev_start_year}-{str(current_start_year)[-2:]}"
    
    # 2) Get team's advanced stats for the entire previous season
    df = stats_getter.getLeagueDashTeamStats(
        team_name,
        prev_season,
        date_from=None,  # No date filtering - get entire season
        date_to=None,    # No date filtering - get entire season
        measure_type="Advanced",
        per_mode="PerGame"
    )
    
    # 3) Return NET_RATING if data exists, otherwise return NaN
    if df.empty:
        return math.nan
    
    return df["NET_RATING"].iloc[0]

def get_last_season_OREB_PCT(team_name: str, season: str) -> float:
    """
    Returns the team's average OREB_PCT for the previous season.
    
    Parameters:
        team_name (str): Full name of the team (e.g., 'Los Angeles Lakers')
        season (str): Current season in format "YYYY-YY" (e.g., "2019-20")
    
    Returns:
        float: Team's OREB_PCT for the previous season, or math.nan if no data found
    """
    # 1) Calculate the previous season
    current_start_year = int(season.split("-")[0])
    prev_start_year = current_start_year - 1
    prev_season = f"{prev_start_year}-{str(current_start_year)[-2:]}"
    
    # 2) Get team's advanced stats for the entire previous season
    df = stats_getter.getLeagueDashTeamStats(
        team_name,
        prev_season,
        date_from=None,  # No date filtering - get entire season
        date_to=None,    # No date filtering - get entire season
        measure_type="Advanced",
        per_mode="PerGame"
    )
    
    # 3) Return OREB_PCT if data exists, otherwise return NaN
    if df.empty:
        return math.nan
    
    return df["OREB_PCT"].iloc[0]

def get_last_season_EFG_PCT(team_name: str, season: str) -> float:
    """
    Returns the team's average EFG_PCT for the previous season.
    
    Parameters:
        team_name (str): Full name of the team (e.g., 'Los Angeles Lakers')
        season (str): Current season in format "YYYY-YY" (e.g., "2019-20")
    
    Returns:
        float: Team's EFG_PCT for the previous season, or math.nan if no data found
    """
    # 1) Calculate the previous season
    current_start_year = int(season.split("-")[0])
    prev_start_year = current_start_year - 1
    prev_season = f"{prev_start_year}-{str(current_start_year)[-2:]}"
    
    # 2) Get team's Four Factors stats for the entire previous season
    df = stats_getter.getLeagueDashTeamStats(
        team_name,
        prev_season,
        date_from=None,  # No date filtering - get entire season
        date_to=None,    # No date filtering - get entire season
        measure_type="Four Factors",
        per_mode="PerGame"
    )
    
    # 3) Return EFG_PCT if data exists, otherwise return NaN
    if df.empty:
        return math.nan
    
    return df["EFG_PCT"].iloc[0]

def get_last_season_TOV_PCT(team_name: str, season: str) -> float:
    """
    Returns the team's average TM_TOV_PCT for the previous season.
    
    Parameters:
        team_name (str): Full name of the team (e.g., 'Los Angeles Lakers')
        season (str): Current season in format "YYYY-YY" (e.g., "2019-20")
    
    Returns:
        float: Team's TM_TOV_PCT for the previous season, or math.nan if no data found
    """
    # 1) Calculate the previous season
    current_start_year = int(season.split("-")[0])
    prev_start_year = current_start_year - 1
    prev_season = f"{prev_start_year}-{str(current_start_year)[-2:]}"
    
    # 2) Get team's Four Factors stats for the entire previous season
    df = stats_getter.getLeagueDashTeamStats(
        team_name,
        prev_season,
        date_from=None,  # No date filtering - get entire season
        date_to=None,    # No date filtering - get entire season
        measure_type="Four Factors",
        per_mode="PerGame"
    )
    
    # 3) Return TM_TOV_PCT if data exists, otherwise return NaN
    if df.empty:
        return math.nan
    
    return df["TM_TOV_PCT"].iloc[0]

def get_last_season_FT_RATE(team_name: str, season: str) -> float:
    """
    Returns the team's average FTA_RATE for the previous season.
    
    Parameters:
        team_name (str): Full name of the team (e.g., 'Los Angeles Lakers')
        season (str): Current season in format "YYYY-YY" (e.g., "2019-20")
    
    Returns:
        float: Team's FTA_RATE for the previous season, or math.nan if no data found
    """
    # 1) Calculate the previous season
    current_start_year = int(season.split("-")[0])
    prev_start_year = current_start_year - 1
    prev_season = f"{prev_start_year}-{str(current_start_year)[-2:]}"
    
    # 2) Get team's Four Factors stats for the entire previous season
    df = stats_getter.getLeagueDashTeamStats(
        team_name,
        prev_season,
        date_from=None,  # No date filtering - get entire season
        date_to=None,    # No date filtering - get entire season
        measure_type="Four Factors",
        per_mode="PerGame"
    )
    
    # 3) Return FTA_RATE if data exists, otherwise return NaN
    if df.empty:
        return math.nan
    
    return df["FTA_RATE"].iloc[0]

def get_netrtg_diff_prev_season(team_name: str, season: str, date_str: str) -> float:
    """
    Returns the difference between current season's NET_RATING (up to date_str) 
    and previous season's NET_RATING.
    
    Formula: Current_Season_NETRTG - Previous_Season_NETRTG
    
    Parameters:
        team_name (str): Full name of the team (e.g., 'Los Angeles Lakers')
        season (str): Current season in format "YYYY-YY" (e.g., "2019-20")
        date_str (str): Date in "MM/DD/YYYY" format for current season cutoff
    
    Returns:
        float: Difference in NET_RATING (current - previous), or math.nan if data unavailable
    """
    # 1) Get current season NET_RATING up to the game date
    target = datetime.strptime(date_str, "%m/%d/%Y")
    cutoff = (target - timedelta(days=1)).strftime("%m/%d/%Y")
    start_yr = int(season.split("-")[0])
    date_from = f"10/01/{start_yr}"
    
    # Get current season NET_RATING from Advanced stats
    df_current = stats_getter.getLeagueDashTeamStats(
        team_name, season, date_from, cutoff,
        measure_type="Advanced", per_mode="PerGame"
    )
    
    if df_current.empty:
        return math.nan
    
    current_netrtg = df_current["NET_RATING"].iloc[0]
    
    # 2) Get previous season NET_RATING using existing function
    prev_netrtg = get_last_season_NETRTG(team_name, season)
    
    if math.isnan(prev_netrtg):
        return math.nan
    
    # 3) Return the difference (current - previous)
    return current_netrtg - prev_netrtg

def get_game_number(team_name: str, season: str, date_str: str) -> int:
    """
    Returns the game number for a team on a given date within a season.
    Game 1 is the first game of the season, Game 2 is the second, etc.
    
    Parameters:
        team_name (str): Full name of the team (e.g., 'Los Angeles Lakers')
        season (str): Season in format "YYYY-YY" (e.g., "2019-20")
        date_str (str): Date in "MM/DD/YYYY" format
    
    Returns:
        int: Game number (1-based), or 0 if no game found on that date
    """
    # 1) Get team ID
    team_id = stats_getter.get_team_id(team_name)
    if team_id is None:
        return 0
    
    # 2) Get full league game log for the season
    df_league = stats_getter.get_league_game_log(season)
    
    # 3) Filter for this team's games and convert dates
    df_team = df_league[df_league["TEAM_ID"] == team_id].copy()
    df_team["GAME_DATE"] = pd.to_datetime(df_team["GAME_DATE"])
    
    # 4) Sort games by date (chronological order)
    df_team = df_team.sort_values("GAME_DATE").reset_index(drop=True)
    
    # 5) Parse target date
    target_date = datetime.strptime(date_str, "%m/%d/%Y")
    
    # 6) Find the game on the target date
    game_mask = df_team["GAME_DATE"] == target_date
    if not game_mask.any():
        return 0  # No game found on this date
    
    # 7) Get the index of the game (0-based) and convert to game number (1-based)
    game_index = df_team.index[game_mask][0]
    game_number = game_index + 1
    
    return game_number

def get_roster_ppg_change(team_name: str, curr_season: str, prev_season: str) -> float:
    """
    Returns the net PPG (points per game) change due to roster changes between prev_season and curr_season.
    Only counts players with GP >= 30 and MIN >= 14.

    - Lost: Players on previous roster but not current (use prev_season/team stats)
    - Added: Players on current roster but not previous (use prev_season/league stats)
    """
    import math

    # Get rosters
    curr_roster = stats_getter.getRoster(team_name, curr_season)
    prev_roster = stats_getter.getRoster(team_name, prev_season)

    # Get prev_season player stats for TEAM
    df_team_prev = getLeagueDashPlayerStats(team_name, season=prev_season, measure_type="Base")
    # Get prev_season player stats for entire LEAGUE
    df_league_prev = getLeagueDashPlayerStats(team_name=None, season=prev_season, measure_type="Base")  # You may need to pass None or "" for league-wide

    if df_team_prev is None or df_league_prev is None or df_team_prev.empty or df_league_prev.empty:
        return math.nan

    # Find lost and added players
    lost_players = set(prev_roster) - set(curr_roster)
    added_players = set(curr_roster) - set(prev_roster)

    # Lost players: get PPG from prev TEAM stats
    lost_ppg = df_team_prev[
        (df_team_prev["PLAYER_NAME"].isin(lost_players)) &
        (df_team_prev["GP"] >= 30) &
        (df_team_prev["MIN"] >= 14)
    ]["PTS"].sum()

    # Added players: get PPG from prev LEAGUE stats
    added_ppg = df_league_prev[
        (df_league_prev["PLAYER_NAME"].isin(added_players)) &
        (df_league_prev["GP"] >= 30) &
        (df_league_prev["MIN"] >= 14)
    ]["PTS"].sum()

    # Net change = added - lost
    return added_ppg - lost_ppg


#stats_cache.clear_cache()

#result = get_hardcoded_2014_2015_averages()
#print(result)





# First, let's see what shot categories are available

# Then filter more carefully







Getting fast break points for all 30 teams in 2014-15...
Processing Atlanta Hawks (1/30)...
Cache hit: get_team_id - Using cached data
Cache hit: LeagueDashTeamStats - Using cached data
  Atlanta Hawks: 6.3 fast break points per game
Processing Boston Celtics (2/30)...
  Waiting 2.7s...
Cache hit: get_team_id - Using cached data
Cache hit: LeagueDashTeamStats - Using cached data
  Boston Celtics: 18.8 fast break points per game
Processing Brooklyn Nets (3/30)...
  Waiting 2.7s...
Cache hit: get_team_id - Using cached data
Cache hit: LeagueDashTeamStats - Using cached data
  Brooklyn Nets: 15.8 fast break points per game
Processing Charlotte Hornets (4/30)...
  Waiting 3.0s...
Cache hit: get_team_id - Using cached data
Cache hit: LeagueDashTeamStats - Using cached data
  Charlotte Hornets: 3.9 fast break points per game
Processing Chicago Bulls (5/30)...
  Waiting 3.7s...
Cache hit: get_team_id - Using cached data
Cache hit: LeagueDashTeamStats - Using cached data
  Chicago Bulls: 4.8 f